In [ ]:
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
import tensorflow as tf
from tensorflow import keras
from bs4 import BeautifulSoup
import pickle

## Analyzing our datas

In [ ]:
with open(r'/content/drive/My Drive/archive (1)/annotations/maksssksksss0.xml','rb') as f:
  content = f.read()
soup = BeautifulSoup(content, 'xml')
soup


<?xml version="1.0" encoding="utf-8"?>
<annotation>
<folder>images</folder>
<filename>maksssksksss0.png</filename>
<size>
<width>512</width>
<height>366</height>
<depth>3</depth>
</size>
<segmented>0</segmented>
<object>
<name>without_mask</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<occluded>0</occluded>
<difficult>0</difficult>
<bndbox>
<xmin>79</xmin>
<ymin>105</ymin>
<xmax>109</xmax>
<ymax>142</ymax>
</bndbox>
</object>
<object>
<name>with_mask</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<occluded>0</occluded>
<difficult>0</difficult>
<bndbox>
<xmin>185</xmin>
<ymin>100</ymin>
<xmax>226</xmax>
<ymax>144</ymax>
</bndbox>
</object>
<object>
<name>without_mask</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<occluded>0</occluded>
<difficult>0</difficult>
<bndbox>
<xmin>325</xmin>
<ymin>90</ymin>
<xmax>360</xmax>
<ymax>141</ymax>
</bndbox>
</object>
</annotation>

### The data we need from above xml file are the texts inside name, xmin, ymin, xmax and ymax tags for each object tag.

In [ ]:
objlist = soup.find_all('object')
namelist = [obj.find('name').text for obj in objlist]
coordlist = []
for obj in objlist:
  xmin = int(obj.find('xmin').text)
  ymin = int(obj.find('ymin').text) 
  xmax = int(obj.find('xmax').text) 
  ymax = int(obj.find('ymax').text)
  coordlist.append((xmin, ymin, xmax, ymax))  

In [ ]:
print(namelist)
print(coordlist)

['without_mask', 'with_mask', 'without_mask']
[(79, 105, 109, 142), (185, 100, 226, 144), (325, 90, 360, 141)]


### These are just the datas for one image. Now, we will extract such datas for all the annotations

In [ ]:
coordlist = []
namelist = []
for i in range(853):
  with open(f'/content/drive/My Drive/archive (1)/annotations/maksssksksss{i}.xml', 'rb') as f:
    content = f.read()
  soup = BeautifulSoup(content,'xml')
  objlist = soup.find_all('object')
  namelist.append([obj.find('name').text for obj in objlist])
  for obj in objlist:
    xmin = int(obj.find('xmin').text)
    ymin = int(obj.find('ymin').text) 
    xmax = int(obj.find('xmax').text) 
    ymax = int(obj.find('ymax').text)
    coordlist.append((xmin, ymin, xmax, ymax))  

In [ ]:
print(namelist)
print(coordlist)
print(len(coordlist))
print(len(namelist))

[['without_mask', 'with_mask', 'without_mask'], ['with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask'], ['with_mask', 'with_mask', 'with_mask', 'mask_weared_incorrect'], ['with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask'], ['with_mask'], ['with_mask', 'without_mask', 'without_mask', 'with_mask'], ['with_mask'], ['with_mask', 'with_mask', 'with_mask', 'without_mask'], ['with_mask'], ['mask_weared_incorrect', 'without_mask'], ['with_mask'], ['mask_weared_incorrect', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask'], ['with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'with_mask', 'without_mask', 'with_mask', 'with_mask'], ['with_mask', 'with_mask', 'm

### Now lets work on a single image

In [ ]:
image = cv2.imread("/content/drive/My Drive/archive (1)/images/maksssksksss0.png")
for i in range(3):
  img = image[coordlist[i][1]:coordlist[i][3], coordlist[i][0]:coordlist[i][2]]
  print(img.shape)
  img = cv2.resize(img, (224,224))
  print(img.shape)
  print(type(img))

(37, 30, 3)
(224, 224, 3)
<class 'numpy.ndarray'>
(44, 41, 3)
(224, 224, 3)
<class 'numpy.ndarray'>
(51, 35, 3)
(224, 224, 3)
<class 'numpy.ndarray'>


### The above example represents the face of a person with and without mask for image0

### Now, lets do this for all the images

In [ ]:
count = 0
imagelist = []
for i in range(853):
  image = cv2.imread(f"/content/drive/My Drive/archive (1)/images/maksssksksss{i}.png")
  for j in range(len(namelist[i])):
    img = image[coordlist[count][1]:coordlist[count][3], coordlist[count][0]:coordlist[count][2]]
    img = cv2.resize(img, (224,224))
    imagelist.append(img)
    count += 1


In [ ]:
len(imagelist)

4072

### Now we will encode the namelist using LabelEncoder

In [ ]:
label = []
from sklearn.preprocessing import LabelEncoder
labelenc = LabelEncoder()
for i in range(len(namelist)):
  for j in range(len(namelist[i])):
    label.append(namelist[i][j])
labels = labelenc.fit_transform(np.array(label))

In [ ]:
print(np.unique(labels))
print(labels.shape)
np.unique(np.array(label))

[0 1 2]
(4072,)


array(['mask_weared_incorrect', 'with_mask', 'without_mask'], dtype='<U21')

### We have encoded mask_weared_incorrect with 0, with_mask with 1 and without_mask with 2 values

In [ ]:
# Saving the x and y values
x = np.array(imagelist)
y = labels
with open("/content/drive/My Drive/all models/face-mask detection/x.pkl", 'wb') as f:
  pickle.dump(x, f)

with open("/content/drive/My Drive/all models/face-mask detection/y.pkl", 'wb') as f:
  pickle.dump(y ,f)


### We will now split the x and y values into training and testing sets

In [ ]:
with open("/content/drive/My Drive/all models/face-mask detection/x.pkl", 'rb') as f:
  x = pickle.load(f)

with open("/content/drive/My Drive/all models/face-mask detection/y.pkl", 'rb') as f:
  y = pickle.load(f)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x.shape

(3257, 224, 224, 3)

In [ ]:
train_y.shape

(3257,)

## Normalizing the data

In [ ]:
train_x = train_x/255
test_x = test_x/255

## Creating a model

In [ ]:
model = keras.Sequential([keras.layers.AveragePooling2D(pool_size=4, strides=2, input_shape=(224,224,3)),
                          keras.layers.Conv2D(64, 4, strides=2, activation='relu'),
                          keras.layers.Conv2D(64, 4, strides=2, activation='relu'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(128, activation='relu'),
                          keras.layers.Dense(3, activation='softmax')
                          
])

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.fit(train_x, train_y, epochs=10, batch_size=4)

Epoch 1/10
815/815 [==============================] - 4s 5ms/step - loss: 0.3360 - accuracy: 0.9021
Epoch 2/10
815/815 [==============================] - 4s 5ms/step - loss: 0.2937 - accuracy: 0.9229
Epoch 3/10
815/815 [==============================] - 4s 5ms/step - loss: 0.2051 - accuracy: 0.9361
Epoch 4/10
815/815 [==============================] - 4s 5ms/step - loss: 0.1920 - accuracy: 0.9383
Epoch 5/10
815/815 [==============================] - 4s 5ms/step - loss: 0.1858 - accuracy: 0.9395
Epoch 6/10
815/815 [==============================] - 4s 4ms/step - loss: 0.1633 - accuracy: 0.9500
Epoch 7/10
815/815 [==============================] - 4s 5ms/step - loss: 0.1555 - accuracy: 0.9469
Epoch 8/10
815/815 [==============================] - 4s 4ms/step - loss: 0.1402 - accuracy: 0.9539
Epoch 9/10
815/815 [==============================] - 4s 4ms/step - loss: 0.1273 - accuracy: 0.9570
Epoch 10/10
815/815 [==============================] - 4s 4ms/step - loss: 0.1173 - accuracy: 0.9638

In [ ]:
model.evaluate(test_x, test_y)

26/26 [==============================] - 0s 12ms/step - loss: 0.2278 - accuracy: 0.9411


[0.22775712609291077, 0.9411042928695679]

In [ ]:
model.save('/content/drive/My Drive/all models/face_mask detection/face_mask')
model.save_weights('/content/drive/My Drive/all models/face_mask detection/face_mask_w')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/drive/My Drive/all models/face_mask detection/face_mask/assets
